# Azure AI Search Multimodal Retrieval Demo

## Introduction
In this demo, we will show you how to create a multimodal (text + images) vector index in Azure AI Search.

**This unofficial code sample was created for an Ignite 2024 demo. It's offered "as-is" and might not work for all customers and all scenarios.**

## Prerequisites
- 🐍 Python 3.9 or higher
- ☁️ Azure Blob Storage
- 🔗 Azure AI Vision Service or Azure AI Multi-Service Account
- 🔗 Azure AI Search Service

## Features Covered
This demo covers the following features:
- ✅ Stored=False
- ✅ Scalar Quantization to int8
- ✅ Reranking w/full precision vectors
- ✅ Oversampling
- ✅ Integrated Vectorization
- ✅ Multi-Vector Search
- ✅ Generate Dense Captions with AI Vision Image Analysis API
- ✅ **[NEW]** Azure AI Vision Embedding Skill
- ✅ **[NEW]** Azure AI Vision Vectorizer
- ✅ **[NEW]** Azure AI Vision Latest Multilingual Model
- ✅ **[NEW]** Vector Weighting

Let's get started!

In [32]:
! pip install azure-search-documents --pre --quiet
! pip install openai python-dotenv azure-identity cohere azure-ai-vision-imageanalysis --quiet

## Generate JSON with text and images via Azure AI Vision Studio
https://portal.vision.cognitive.azure.com/demo/image-captioning

In [34]:
import json
import os
from uuid import uuid4

from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient

# Get environment variables for Azure AI Vision
try:
    endpoint = os.getenv("AZURE_AI_VISION_ENDPOINT")
    key = os.getenv("AZURE_AI_VISION_API_KEY")
    connection_string = os.getenv("BLOB_CONNECTION_STRING")
    # container_name = os.getenv("BLOB_CONTAINER_NAME")
    container_name = "vector-sandbox"
except KeyError as e:
    print(f"Missing environment variable: {str(e)}")
    print("Set them before running this sample.")
    exit()

# Create an Image Analysis client
client = ImageAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(key))

# Setup for Azure Blob Storage
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

def get_caption(image_url):
    """
    Get a caption for the image using Azure AI Vision.
    """
    try:
        result = client.analyze_from_url(
            image_url=image_url,
            visual_features=[VisualFeatures.CAPTION, VisualFeatures.READ],
            gender_neutral_caption=False
        )
        if result.caption is not None:
            return result.caption.text
        else:
            return "No caption available"
    except Exception as e:
        print(f"An error occurred: {e}")
        return "Error generating caption"

def generate_json_objects():
    json_objects = []

    # Iterate over the blobs in the container
    for blob in container_client.list_blobs():
        image_url = f"https://{blob_service_client.account_name}.blob.core.windows.net/{container_name}/{blob.name}"
        caption = get_caption(image_url)

        json_object = {"id": str(uuid4()), "imageUrl": image_url, "caption": caption}
        json_objects.append(json_object)

    return json_objects

def write_to_file(json_objects):
    # Write the updated JSON to a file
    with open("build-demo.json", "w") as json_file:
        json.dump(json_objects, json_file, indent=4)

json_objects = generate_json_objects()
write_to_file(json_objects)

In [126]:
import os

from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.indexes.models import (
    AIServicesVisionParameters,
    AIServicesVisionVectorizer,
    AIStudioModelCatalogName,
    AzureMachineLearningVectorizer,
    AzureOpenAIVectorizer,
    AzureOpenAIModelName,
    AzureOpenAIParameters,
    BlobIndexerDataToExtract,
    BlobIndexerParsingMode,
    CognitiveServicesAccountKey,
    DefaultCognitiveServicesAccount,
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    FieldMapping,
    HnswAlgorithmConfiguration,
    HnswParameters,
    IndexerExecutionStatus,
    IndexingParameters,
    IndexingParametersConfiguration,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    ScalarQuantizationCompressionConfiguration,
    ScalarQuantizationParameters,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchIndexerDataIdentity,
    SearchIndexerDataSourceConnection,
    SearchIndexerSkillset,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
    VisionVectorizeSkill
)
from azure.search.documents.models import (
    HybridCountAndFacetMode,
    HybridSearch,
    SearchScoreThreshold,
    VectorizableTextQuery,
    VectorizableImageBinaryQuery,
    VectorizableImageUrlQuery,
    VectorSimilarityThreshold,
)
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
from IPython.display import Image, display, HTML
from openai import AzureOpenAI

In [127]:
# Load environment variables
load_dotenv()

# Configuration
AZURE_AI_VISION_API_KEY = os.getenv("AZURE_AI_VISION_API_KEY")
AZURE_AI_VISION_ENDPOINT = os.getenv("AZURE_AI_VISION_ENDPOINT")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
BLOB_CONNECTION_STRING = os.getenv("BLOB_CONNECTION_STRING")
BLOB_CONTAINER_NAME = os.getenv("BLOB_CONTAINER_NAME")
INDEX_NAME = "build-multimodal-demo"
SEARCH_SERVICE_API_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")
SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")

In [128]:
# User-specified parameter
USE_AAD_FOR_SEARCH = True  # Set this to False to use API key for authentication

def authenticate_azure_search(api_key=None, use_aad_for_search=False):
    if use_aad_for_search:
        print("Using AAD for authentication.")
        credential = DefaultAzureCredential()
    else:
        print("Using API keys for authentication.")
        if api_key is None:
            raise ValueError("API key must be provided if not using AAD for authentication.")
        credential = AzureKeyCredential(api_key)
    return credential

azure_search_credential = authenticate_azure_search(api_key=SEARCH_SERVICE_API_KEY, use_aad_for_search=USE_AAD_FOR_SEARCH)


Using AAD for authentication.


## Create a blob data source connector on Azure AI Search

In [129]:
def create_or_update_data_source(indexer_client, container_name, connection_string, index_name):
    """
    Create or update a data source connection for Azure AI Search.
    """
    container = SearchIndexerDataContainer(name=container_name)
    data_source_connection = SearchIndexerDataSourceConnection(
        name=f"{index_name}-blob",
        type="azureblob",
        connection_string=connection_string,
        container=container
    )
    try:
        indexer_client.create_or_update_data_source_connection(data_source_connection)
        print(f"Data source '{index_name}-blob' created or updated successfully.")
    except Exception as e:
        raise Exception(f"Failed to create or update data source due to error: {e}")

# Create a SearchIndexerClient instance
indexer_client = SearchIndexerClient(SEARCH_SERVICE_ENDPOINT, azure_search_credential)

# Call the function to create or update the data source
create_or_update_data_source(indexer_client, BLOB_CONTAINER_NAME, BLOB_CONNECTION_STRING, INDEX_NAME)

Data source 'build-multimodal-demo-blob' created or updated successfully.


## Create a search index

In [130]:
def create_fields():
    """Creates the fields for the search index based on the specified schema."""
    return [
        SimpleField(
            name="id", type=SearchFieldDataType.String, key=True, filterable=True
        ),
        SearchField(name="caption", type=SearchFieldDataType.String, searchable=True),
        SearchField(name="imageUrl", type=SearchFieldDataType.String, searchable=True),
        SearchField(
            name="captionVector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            vector_search_dimensions=1024,
            vector_search_profile_name="myHnswProfile",
            stored=False,
        ),
        SearchField(
            name="imageVector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            vector_search_dimensions=1024,
            vector_search_profile_name="myHnswProfile",
            stored=False,
        ),
    ]


def create_vector_search_configuration():
    """Creates the vector search configuration."""
    return VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="myHnsw",
                parameters=HnswParameters(
                    m=4,
                    ef_construction=400,
                    ef_search=500,
                    metric=VectorSearchAlgorithmMetric.COSINE,
                ),
            )
        ],
        compressions=[
            ScalarQuantizationCompressionConfiguration(
                name="myScalarQuantization",
                rerank_with_original_vectors=True,
                default_oversampling=10,
                parameters=ScalarQuantizationParameters(quantized_data_type="int8"),
            )
        ],
        vectorizers=[
            AIServicesVisionVectorizer(
                name="myAIServicesVectorizer",
                kind="aiServicesVision",
                ai_services_vision_parameters=AIServicesVisionParameters(
                    model_version="2023-04-15",
                    resource_uri=AZURE_AI_VISION_ENDPOINT,
                    api_key=AZURE_AI_VISION_API_KEY,
                ),
            )
        ],
        profiles=[
            VectorSearchProfile(
                name="myHnswProfile",
                algorithm_configuration_name="myHnsw",
                compression_configuration_name="myScalarQuantization",
                vectorizer="myAIServicesVectorizer",
            )
        ],
    )


def create_search_index(index_client, index_name, fields, vector_search):
    """Creates or updates a search index."""
    index = SearchIndex(
        name=index_name,
        fields=fields,
        vector_search=vector_search,
    )
    index_client.create_or_update_index(index=index)


index_client = SearchIndexClient(
    endpoint=SEARCH_SERVICE_ENDPOINT, credential=azure_search_credential
)
fields = create_fields()
vector_search = create_vector_search_configuration()

# Create the search index with the adjusted schema
create_search_index(index_client, INDEX_NAME, fields, vector_search)
print(f"Created index: {INDEX_NAME}")

Created index: build-multimodal-demo


## Create a Skillset    

In [131]:
def create_text_embedding_skill():
    return VisionVectorizeSkill(
        name="text-embedding-skill",
        description="Skill to generate embeddings for text via Azure AI Vision",
        context="/document",
        model_version="2023-04-15",
        inputs=[InputFieldMappingEntry(name="text", source="/document/caption")],
        outputs=[OutputFieldMappingEntry(name="vector", target_name="captionVector")],
    )

def create_image_embedding_skill():
    return VisionVectorizeSkill(
        name="image-embedding-skill",
        description="Skill to generate embeddings for image via Azure AI Vision",
        context="/document",
        model_version="2023-04-15",
        inputs=[InputFieldMappingEntry(name="url", source="/document/imageUrl")],
        outputs=[OutputFieldMappingEntry(name="vector", target_name="imageVector")],
    )

def create_skillset(client, skillset_name, text_embedding_skill, image_embedding_skill):
    skillset = SearchIndexerSkillset(
        name=skillset_name,
        description="Skillset for generating embeddings",
        skills=[text_embedding_skill, image_embedding_skill],
        cognitive_services_account=CognitiveServicesAccountKey(
            key=AZURE_AI_VISION_API_KEY,
            description="AI Vision Multi Service Account in West US",
        ),
    )
    client.create_or_update_skillset(skillset)

client = SearchIndexerClient(
    endpoint=SEARCH_SERVICE_ENDPOINT, credential=azure_search_credential
)
skillset_name = f"{INDEX_NAME}-skillset"
text_embedding_skill = create_text_embedding_skill()
image_embedding_skill = create_image_embedding_skill()

create_skillset(client, skillset_name, text_embedding_skill, image_embedding_skill)
print(f"Created skillset: {skillset_name}")

Created skillset: build-multimodal-demo-skillset


## Run Indexer

In [132]:
def create_and_run_indexer(indexer_client, indexer_name, skillset_name, index_name, data_source_name):
    indexer = SearchIndexer(
        name=indexer_name,
        description="Indexer to index documents and generate embeddings",
        skillset_name=skillset_name,
        target_index_name=index_name,
        data_source_name=data_source_name,
        parameters=IndexingParameters(
            configuration=IndexingParametersConfiguration(
                parsing_mode=BlobIndexerParsingMode.JSON_ARRAY,
                query_timeout=None,
            ),
        ),
        field_mappings=[FieldMapping(source_field_name="id", target_field_name="id")],
        output_field_mappings=[
            FieldMapping(source_field_name="/document/captionVector", target_field_name="captionVector"),
            FieldMapping(source_field_name="/document/imageVector", target_field_name="imageVector"),
        ],
    )

    indexer_client.create_or_update_indexer(indexer)
    print(f"{indexer_name} created or updated.")

    indexer_client.run_indexer(indexer_name)
    print(f"{indexer_name} is running. If queries return no results, please wait a bit and try again.")

indexer_client = SearchIndexerClient(
    endpoint=SEARCH_SERVICE_ENDPOINT, credential=azure_search_credential
)
data_source_name = f"{INDEX_NAME}-blob"
indexer_name = f"{INDEX_NAME}-indexer"

create_and_run_indexer(indexer_client, indexer_name, skillset_name, INDEX_NAME, data_source_name)

build-multimodal-demo-indexer created or updated.
build-multimodal-demo-indexer is running. If queries return no results, please wait a bit and try again.


## Simple vector search (text to text)

In [137]:
# Initialize the SearchClient
search_client = SearchClient(
    SEARCH_SERVICE_ENDPOINT,
    index_name=INDEX_NAME,
    credential=azure_search_credential,
)

# Define the query
# query = "sunglasses for holiday"
# query = "休日のサングラス" # Japanese query
query = "female red dresses" # Spanish query

vector_query = VectorizableTextQuery(
    text=query,
    k_nearest_neighbors=3,
    fields="captionVector",
    # fields="imageVector",
)

# Perform the search
results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    top=3
)


# Print the results
for result in results:
    print(f"Caption: {result['caption']}")
    print(f"Score: {result['@search.score']}")
    display(HTML(f'<img src="{result["imageUrl"]}" style="width:200px;"/>'))
    print("-" * 50) 

Caption: a woman in a red dress
Score: 0.9404777


--------------------------------------------------
Caption: a red handbag with a strap
Score: 0.90242684


--------------------------------------------------
Caption: a woman in a dress
Score: 0.8996681


--------------------------------------------------


## Simple Vector Search (text to image)

In [134]:
# Define the text query
query = "shoes for running"
text_vector_query = VectorizableTextQuery(
    text=query,
    k_nearest_neighbors=10,
    fields="captionVector",
)
# Define the image query
image_vector_query = VectorizableImageUrlQuery(  # Alternatively, use VectorizableImageBinaryQuery
    url="https://images.unsplash.com/photo-1542291026-7eec264c27ff?q=80&w=1770&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D",  # Image of a Red Nike Running Shoe
    k_nearest_neighbors=10,
    fields="imageVector",
)

# Perform the search
results = search_client.search(
    search_text=None, vector_queries=[text_vector_query, image_vector_query], top=3
)

# Print the results
for result in results:
    print(f"Caption: {result['caption']}")
    print(f"Score: {result['@search.score']}")
    print(f"URL: {result['imageUrl']}")
    display(HTML(f'<img src="{result["imageUrl"]}" style="width:200px;"/>'))
    print("-" * 50)  

Caption: a group of high heels
Score: 0.032522473484277725
URL: https://fsunavalastorage.blob.core.windows.net/vector-sandbox/Picture504.jpg


--------------------------------------------------
Caption: a group of clothes and shoes
Score: 0.01666666753590107
URL: https://fsunavalastorage.blob.core.windows.net/vector-sandbox/Picture477.jpg


--------------------------------------------------
Caption: a group of shoes on a wood surface
Score: 0.01666666753590107
URL: https://fsunavalastorage.blob.core.windows.net/vector-sandbox/Picture496.jpg


--------------------------------------------------


## Multi-modal vector search with weighting images 100x more than captions

In [114]:
# Define the text query
query = "shoes for running"
text_vector_query = VectorizableTextQuery(
    text=query,
    k_nearest_neighbors=5,
    fields="captionVector",
)
# Define the image query
image_vector_query = VectorizableImageUrlQuery(  # Alternatively, use VectorizableImageBinaryQuery
    url="https://images.unsplash.com/photo-1542291026-7eec264c27ff?q=80&w=1770&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D",  # Image of a Red Nike Running Shoe
    k_nearest_neighbors=5,
    fields="imageVector",
    weight=100,
)

# Perform the search
results = search_client.search(
    search_text=None, vector_queries=[text_vector_query, image_vector_query], top=3
)

# Print the results
for result in results:
    print(f"Caption: {result['caption']}")
    print(f"Score: {result['@search.score']}")
    print(f"URL: {result['imageUrl']}")
    display(HTML(f'<img src="{result["imageUrl"]}" style="width:200px;"/>'))
    print("-" * 50)  

Caption: a group of shoes on a wood surface
Score: 1.6666667461395264
URL: https://fsunavalastorage.blob.core.windows.net/vector-sandbox/Picture496.jpg


--------------------------------------------------
Caption: a woman in blue running
Score: 1.6393442153930664
URL: https://fsunavalastorage.blob.core.windows.net/vector-sandbox/Picture654.jpg


--------------------------------------------------
Caption: a group of high heels
Score: 1.6292965412139893
URL: https://fsunavalastorage.blob.core.windows.net/vector-sandbox/Picture504.jpg


--------------------------------------------------
